<a href="https://colab.research.google.com/github/lauberto/politopic/blob/main/notebook/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modelling on Politicians' speech

This notebook shows how to apply topic modelling on the transcription of yt videos with Politicians' speech. The transcription is performed with [`whisper`](https://github.com/openai/whisper).

## Installing dependencies

In [1]:
!sudo apt update && sudo apt install ffmpeg

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd6

In [2]:
# Installing whisper
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gffm5jc4
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-gffm5jc4


 Installing [`pytube`](https://https://github.com/pytube/pytube). and downloading the [video](https://https://www.youtube.com/watch?v=knj8ULToNvo) from youtube.

In [3]:
!pip install git+https://github.com/pytube/pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytube/pytube to /tmp/pip-req-build-kk78m3ia
  Running command git clone -q https://github.com/pytube/pytube /tmp/pip-req-build-kk78m3ia


## First Demo - Salvini's Speech at a Coldiretti Conference.

Who's better than a populist far-right politician like Salvini to start with the analysis of political speeches?

In [4]:
video_ids = ['knj8ULToNvo', 'wVHyFwyhQwE&t=73s']
video_urls = ['https://youtu.be/' + video_id for video_id in video_ids]

### Download & Transcription
Download the video from YouTube and transcribe it using whisper. We are going to 

In [5]:
import whisper
from pytube import YouTube

In [7]:
whisper_model = whisper.load_model('base')

100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 29.0MiB/s]


In [14]:
titles = []
texts = []

for url in video_urls:
  yt = YouTube(url=url)
  titles.append(yt.title)
  path = yt.streams.filter(only_audio=True)[0].download(filename="audio.mp4")
  transcription = whisper_model.transcribe(path)
  texts.append(transcription["text"])  

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


### Sentence segmentation

In [29]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
from nltk import sent_tokenize

In [31]:
docs = []
video_titles = []

for title, text in zip(titles, texts):
  sents = sent_tokenize(text)
  docs.extend(sents)
  video_titles.extend([title]*len(sents))

Let's save the texts on google drive just in case.

In [33]:
import pandas as pd

df = pd.DataFrame({"Title": video_titles, "Text": docs})
df.head()

,Title,Text
0,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),accogliamo un ultimo dei nostri amici.
1,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Parlaramo poco fammi, ha detto, sono una perso..."
2,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),Quindi diamo la parola per ultimo proprio il s...
3,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Cioè, presentazione alla Gesmundo New Look, se..."
4,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Non ho il compitino pronto, cioè il mio uffici..."


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
df.to_csv('/content/drive/MyDrive/Data/political_speeches/politopic_salvini.tsv', sep='\t', index=False)

## Latent Dirichlet Allocation